In [0]:
df = spark.read.format('delta').load('s3://silverbucketkrsna/Retail_YT_Project/orders/')

In [0]:
dimCustomerDf = spark.sql('select DimCustomerKey, customer_id as dim_customer_id from databricks_cata.gold.dimcustomers')
dimProductsDf = spark.sql('select product_id as DimProductKey, product_id as dim_product_id from databricks_cata.gold.dimproducts')

In [0]:
# dimCustomerDf.limit(5).display()
# dimProductsDf.limit(5).display()

In [0]:
factDf = df.join(dimCustomerDf, df['customer_id'] == dimCustomerDf['dim_customer_id'], how = 'left')\
           .join(dimProductsDf, df['customer_id'] == dimProductsDf['dim_product_id'], how = 'left')

newfactDf = factDf.drop('dim_customer_id', 'dim_product_id')
# newfactDf.limit(5).display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('databricks_cata.gold.factOrders'):

    dlt_obj = DeltaTable.forName(spark, 'databricks_cata.gold.factOrders')
    dlt_obj.alias('tgt').merge(newfactDf.alias('src'), 'tgt.order_id = src.order_id and tgt.dimCustomerKey = src.dimCustomerKey and tgt.DimProductKey = src.DimProductKey')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    newfactDf.write.format('delta').mode('overwrite').saveAsTable('databricks_cata.gold.factOrders')